# Currency Price Prediction - Model Training

This notebook trains LSTM models for currency price prediction and saves them to Google Drive.

**Features:**
- GPU acceleration support
- Google Drive integration for model persistence
- Progress tracking and visualization
- Batch training for all currencies

## 1. Setup Environment

In [ ]:
# Check GPU availability
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

if tf.config.list_physical_devices('GPU'):
    print("\n✓ GPU detected! Training will be accelerated.")
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"  Device: {gpu}")
else:
    print("\n⚠ No GPU detected. Training will use CPU (slower).")
    print("  To enable GPU: Runtime > Change runtime type > Hardware accelerator > GPU")

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create directory for models in Drive
drive_model_dir = '/content/drive/MyDrive/CurrencyCapPortal/models'
os.makedirs(drive_model_dir, exist_ok=True)
print(f"\n✓ Models will be saved to: {drive_model_dir}")

## 3. Clone Repository

In [ ]:
# Clone your repository (update with your repo URL)
# Option A: Clone from GitHub
!git clone https://github.com/YOUR_USERNAME/CurrencyCapPortal.git
%cd CurrencyCapPortal

# Option B: If already cloned or using manual upload, just navigate
# import os
# if os.path.exists('CurrencyCapPortal'):
#     %cd CurrencyCapPortal
# else:
#     print("Repository not found. Please upload files manually or update GitHub URL above.")

# Verify the ml module exists
import os
if os.path.exists('ml/predictor.py'):
    print("✓ Found ml/predictor.py")
else:
    print("✗ ml/predictor.py not found!")
    print("Current directory:", os.getcwd())
    print("\nPlease either:")
    print("1. Update the GitHub URL above and re-run this cell")
    print("2. Or use manual file upload (see COLAB_SETUP.md)")

In [ ]:
# # MANUAL UPLOAD OPTION - Uncomment to use
# from google.colab import files
# import os
# import zipfile

# print("Upload your CurrencyCapPortal project as a ZIP file")
# print("(Right-click your project folder > Compress to create a zip)")
# print()

# uploaded = files.upload()

# # Extract the zip
# for filename in uploaded.keys():
#     print(f"Extracting {filename}...")
#     !unzip -q {filename}
    
# # Navigate to the project directory
# project_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d != 'sample_data']
# if project_dirs:
#     %cd {project_dirs[0]}
#     print(f"✓ Navigated to {project_dirs[0]}")
#     !ls -la
# else:
#     print("✗ Could not find project directory")

## 3A. Alternative: Manual File Upload

If you don't have your code on GitHub, uncomment and run this cell instead of Cell 3:

## 4. Install Dependencies

In [ ]:
# Install required packages (suppress dependency warnings)
!pip install -q -r requirements.txt 2>&1 | grep -v "dependency conflicts" || true
!pip install -q jedi  # Fix ipython autocomplete dependency

print("✓ Dependencies installed")

## 5. Training Functions

In [ ]:
import json
import shutil
from datetime import datetime
from ml.predictor import CurrencyPredictor

# Configuration
DRIVE_MODEL_DIR = '/content/drive/MyDrive/CurrencyCapPortal/models'
LOCAL_MODEL_DIR = 'models'

def train_single_currency(currency_code: str, epochs: int = 100, save_to_drive: bool = True):
    """Train model for a single currency and optionally save to Drive."""
    print(f"\n{'='*60}")
    print(f"Training model for {currency_code.upper()}")
    print(f"{'='*60}\n")
    
    predictor = CurrencyPredictor(currency_code=currency_code)
    
    try:
        # Train the model
        metrics = predictor.train_model(epochs=epochs)
        
        # Save to Google Drive
        if save_to_drive:
            local_model_path = f'{LOCAL_MODEL_DIR}/{currency_code}_model.keras'
            drive_model_path = f'{DRIVE_MODEL_DIR}/{currency_code}_model.keras'
            
            if os.path.exists(local_model_path):
                shutil.copy2(local_model_path, drive_model_path)
                print(f"✓ Model saved to Google Drive: {drive_model_path}")
        
        print(f"\n✓ Successfully trained {currency_code.upper()} model")
        print(f"  Final Loss: {metrics.get('loss', 'N/A'):.4f}" if isinstance(metrics.get('loss'), float) else "")
        
        return True, metrics
    except Exception as e:
        print(f"\n✗ Failed to train {currency_code.upper()} model: {e}")
        return False, None


def train_all_currencies(epochs: int = 100, save_to_drive: bool = True):
    """Train models for all available currencies."""
    with open('api/latest.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    currencies = [item['ab'] for item in data]
    print(f"Found {len(currencies)} currencies to train")
    print(f"Epochs per currency: {epochs}\n")
    
    results = {}
    training_log = []
    
    start_time = datetime.now()
    
    for idx, currency in enumerate(currencies, 1):
        print(f"\n[{idx}/{len(currencies)}] Processing {currency.upper()}...")
        
        success, metrics = train_single_currency(currency, epochs, save_to_drive)
        results[currency] = success
        
        training_log.append({
            'currency': currency,
            'success': success,
            'metrics': metrics,
            'timestamp': datetime.now().isoformat()
        })
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    # Print summary
    print(f"\n{'='*60}")
    print("Training Summary")
    print(f"{'='*60}")
    successful = sum(1 for v in results.values() if v)
    print(f"Successful: {successful}/{len(results)}")
    print(f"Failed: {len(results) - successful}/{len(results)}")
    print(f"Total time: {duration}")
    
    if successful < len(results):
        print("\nFailed currencies:")
        for currency, success in results.items():
            if not success:
                print(f"  - {currency}")
    
    # Save training log to Drive
    if save_to_drive:
        log_path = f'{DRIVE_MODEL_DIR}/training_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
        with open(log_path, 'w') as f:
            json.dump(training_log, f, indent=2)
        print(f"\n✓ Training log saved to: {log_path}")
    
    return results, training_log


print("✓ Training functions loaded")

## 6. Train Single Currency

Use this cell to train a model for a specific currency.

In [ ]:
# Configure training
CURRENCY = 'usd'  # Change this to your desired currency
EPOCHS = 100       # Adjust number of epochs

# Train the model
success, metrics = train_single_currency(CURRENCY, epochs=EPOCHS, save_to_drive=True)

if success:
    print(f"\n🎉 Training completed successfully!")
    print(f"Model saved to Google Drive")

## 7. Train All Currencies (Batch Mode)

⚠️ **Warning:** This will train models for ALL currencies, which may take several hours.

Make sure you have:
- GPU enabled
- Sufficient Colab runtime (consider Colab Pro for longer sessions)

In [ ]:
# Configure batch training
EPOCHS = 100  # Epochs per currency

# Train all currencies
results, training_log = train_all_currencies(epochs=EPOCHS, save_to_drive=True)

print("\n🎉 Batch training completed!")
print("All models saved to Google Drive")

## 8. Visualize Training Results (Optional)

In [ ]:
import matplotlib.pyplot as plt

# Example: Visualize training history for a specific currency
CURRENCY = 'usd'

predictor = CurrencyPredictor(currency_code=CURRENCY)

# You can add visualization code here if your predictor stores history
print(f"Visualization for {CURRENCY.upper()}")
print("Add custom visualization code based on your training metrics")

## 9. Download Models to Local Machine (Optional)

If you want to download trained models directly to your computer:

In [ ]:
from google.colab import files
import zipfile

# Zip all models
!zip -r models.zip models/

# Download the zip file
files.download('models.zip')
print("✓ Models downloaded")

## Notes

### GPU Usage
- Enable GPU: `Runtime > Change runtime type > Hardware accelerator > GPU`
- Free tier: T4 GPU (limited hours per day)
- Colab Pro: More GPU time + better GPUs (V100, A100)

### Model Persistence
- Models are automatically saved to Google Drive
- Location: `/MyDrive/CurrencyCapPortal/models/`
- Training logs include timestamps and metrics

### Memory Management
- Free Colab: ~12GB RAM
- If you run out of memory, try:
  - Reducing batch size in your model
  - Training fewer currencies at once
  - Restarting runtime between training sessions

### Session Timeout
- Free Colab: 12 hours max, 90 min idle timeout
- Keep browser tab active to prevent disconnection
- Consider Colab Pro for 24-hour sessions